<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/Hospitalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hospitalization

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

## Load Data

In [ ]:
hospital_data_county = pd.read_csv('https://data.ca.gov/dataset/529ac907-6ba1-4cb7-9aae-8966fc96aeef/resource/42d33765-20fd-44b8-a978-b083b7542225/download/hospitals_by_county.csv')
hospital_data_county.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
0,Yuba,2020-03-29,2.0,12.0,NaN,NaN,2.0,4.0,2.0
1,Stanislaus,2020-03-29,20.0,48.0,NaN,NaN,9.0,8.0,25.0
2,Riverside,2020-03-29,44.0,78.0,NaN,NaN,13.0,20.0,46.0
3,Napa,2020-03-29,0.0,26.0,NaN,NaN,NaN,NaN,5.0
4,Marin,2020-03-29,7.0,13.0,NaN,NaN,2.0,6.0,11.0


In [ ]:
hospital_data_county.shape

(7781, 9)

## Clean Data

In [ ]:
hospital_data_county_notna = hospital_data_county[hospital_data_county['hospitalized_covid_patients'].notna()]

In [ ]:
hospital_data_county_notna.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
1285,San Mateo,2020-04-21,53.0,14.0,67.0,395.0,19.0,2.0,33.0
1286,Contra Costa,2020-04-21,33.0,44.0,77.0,1037.0,13.0,11.0,71.0
1287,Napa,2020-04-21,1.0,1.0,2.0,387.0,1.0,0.0,9.0
1288,Tuolumne,2020-04-21,0.0,0.0,0.0,90.0,0.0,0.0,3.0
1289,Ventura,2020-04-21,28.0,28.0,56.0,976.0,9.0,10.0,25.0


In [ ]:
hospital_data_county_notna.shape

(6496, 9)

## Pre Process Data

In [ ]:
hospital_data_county.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
0,Yuba,2020-03-29,2.0,12.0,NaN,NaN,2.0,4.0,2.0
1,Stanislaus,2020-03-29,20.0,48.0,NaN,NaN,9.0,8.0,25.0
2,Riverside,2020-03-29,44.0,78.0,NaN,NaN,13.0,20.0,46.0
3,Napa,2020-03-29,0.0,26.0,NaN,NaN,NaN,NaN,5.0
4,Marin,2020-03-29,7.0,13.0,NaN,NaN,2.0,6.0,11.0


In [ ]:
ignore_columns = ['county', 'todays_date', 'hospitalized_covid_patients', 'icu_covid_confirmed_patients', 'icu_suspected_covid_patients', 'icu_available_beds']
percent_suspected_covid_patients_positive = 1.00
complete = 0
hospital_data_county['Percent_positive_COVID_Hospital_BED'] = 0
hospital_data_county['beds_occupied'] = 0

Formula Used:

beds occupied = hospitalized_covid_patient + hospitalized_suspected_covid_patient + hospitalized_covid_confirmed_patients

Percent_positive_COVID_Hospital_BED = hospitalized_covid_confirmed_patients + (hospitalized_suspected_covid_patients * *percent that are actually positive*) / all_hospital_beds

In [ ]:
for index, row in hospital_data_county.iterrows():
  complete = (index / len(hospital_data_county)) * 100
  if (complete % 5 == 0):
    print("Percent Complete:", complete)
  
  for column in hospital_data_county.columns:
    if (column == 'hospitalized_covid_confirmed_patients'):
      hospitalized_covid_confirmed_patients = hospital_data_county[column][index]
    elif (column == 'hospitalized_suspected_covid_patients'):
      hospitalized_suspected_covid_patients = hospital_data_county[column][index]
    elif (column == 'hospitalized_covid_patients'):
      hospitalized_covid_patients = hospital_data_county[column][index]
    elif (column == 'all_hospital_beds'):
      all_hospital_beds = hospital_data_county[column][index]
      try:
        hospital_data_county['Percent_positive_COVID_Hospital_BED'][index] = (hospitalized_covid_confirmed_patients + (hospitalized_suspected_covid_patients * percent_suspected_covid_patients_positive) + hospitalized_covid_patients) / all_hospital_beds
        hospital_data_county['beds_occupied'][index] = hospitalized_covid_confirmed_patients + hospitalized_suspected_covid_patients + hospitalized_covid_patients
      except:
        hospital_data_county['Percent_positive_COVID_Hospital_BED'][index] = 'NaN'
        hospital_data_county['beds_occupied'][index] = hospitalized_covid_confirmed_patients + hospitalized_suspected_covid_patients + hospitalized_covid_patients

print("Percent Complete: 100.00")

Percent Complete: 0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packag

Percent Complete: 100.00


In [ ]:
hospital_data_county.tail()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,Percent_positive_COVID_Hospital_BED,beds_occupied
7776,Stanislaus,2020-08-14,192.0,51.0,243.0,1308.0,55.0,8.0,13.0,0.371560,486.0
7777,Calaveras,2020-08-14,3.0,1.0,4.0,33.0,2.0,0.0,5.0,0.242424,8.0
7778,San Joaquin,2020-08-14,149.0,17.0,166.0,1012.0,54.0,4.0,25.0,0.328063,332.0
7779,El Dorado,2020-08-14,1.0,0.0,1.0,189.0,1.0,0.0,10.0,0.010582,2.0
7780,Placer,2020-08-14,53.0,22.0,75.0,740.0,12.0,2.0,28.0,0.202703,150.0


In [ ]:
hospital_data_county['hospital_category'] = 6
for index, row in hospital_data_county.iterrows():
  value = hospital_data_county['Percent_positive_COVID_Hospital_BED'][index]
  if (value > 0.8):
    hospital_data_county['hospital_category'][index] = 5
  elif (value > 0.6 and value <= 0.8):
    hospital_data_county['hospital_category'][index] = 4
  elif (value > 0.4 and value <= 0.6):
    hospital_data_county['hospital_category'][index] = 3
  elif (value > 0.2 and value <= 0.4):
    hospital_data_county['hospital_category'][index] = 2
  else:
    hospital_data_county['hospital_category'][index] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-p

In [ ]:
hospital_data_county.tail()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,Percent_positive_COVID_Hospital_BED,beds_occupied,hospital_category
7776,Stanislaus,2020-08-14,192.0,51.0,243.0,1308.0,55.0,8.0,13.0,0.371560,486.0,2
7777,Calaveras,2020-08-14,3.0,1.0,4.0,33.0,2.0,0.0,5.0,0.242424,8.0,2
7778,San Joaquin,2020-08-14,149.0,17.0,166.0,1012.0,54.0,4.0,25.0,0.328063,332.0,2
7779,El Dorado,2020-08-14,1.0,0.0,1.0,189.0,1.0,0.0,10.0,0.010582,2.0,1
7780,Placer,2020-08-14,53.0,22.0,75.0,740.0,12.0,2.0,28.0,0.202703,150.0,2


## Los Angeles County Data

In [ ]:
hospital_data_county_LA = hospital_data_county[hospital_data_county['county'] == 'Los Angeles']
hospital_data_county_LA.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,Percent_positive_COVID_Hospital_BED,beds_occupied,hospital_category
35,Los Angeles,2020-03-29,489.0,1132.0,NaN,NaN,191.0,182.0,345.0,NaN,NaN,1
96,Los Angeles,2020-03-30,601.0,1277.0,NaN,NaN,245.0,244.0,456.0,NaN,NaN,1
148,Los Angeles,2020-03-31,713.0,1239.0,NaN,NaN,315.0,239.0,445.0,NaN,NaN,1
213,Los Angeles,2020-04-01,739.0,1332.0,NaN,NaN,335.0,220.0,492.0,NaN,NaN,1
247,Los Angeles,2020-04-02,818.0,1270.0,NaN,NaN,346.0,193.0,488.0,NaN,NaN,1


In [ ]:
hospital_data_county_LA.tail()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,Percent_positive_COVID_Hospital_BED,beds_occupied,hospital_category
7546,Los Angeles,2020-08-10,1538.0,446.0,1984.0,19164.0,493.0,58.0,850.0,0.207055,3968.0,2
7558,Los Angeles,2020-08-11,1481.0,470.0,1951.0,19342.0,481.0,66.0,871.0,0.201737,3902.0,2
7636,Los Angeles,2020-08-12,1415.0,472.0,1887.0,19364.0,460.0,56.0,842.0,0.194898,3774.0,1
7702,Los Angeles,2020-08-13,1393.0,520.0,1913.0,19575.0,456.0,72.0,977.0,0.195453,3826.0,1
7729,Los Angeles,2020-08-14,1357.0,556.0,1913.0,19573.0,441.0,75.0,865.0,0.195473,3826.0,1


## Counties with category 5 ranking

In [ ]:
hospital_data_county_category_5 = hospital_data_county[hospital_data_county['hospital_category'] == 5]
hospital_data_county_category_5.head()

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds,Percent_positive_COVID_Hospital_BED,beds_occupied,hospital_category
1312,Sacramento,2020-04-21,54.0,39.0,93.0,50.0,21.0,8.0,72.0,3.720000,186.0,5
1354,Sacramento,2020-04-22,47.0,23.0,70.0,50.0,18.0,5.0,79.0,2.800000,140.0,5
4040,Imperial,2020-06-09,83.0,2.0,85.0,212.0,18.0,0.0,2.0,0.801887,170.0,5
4713,Imperial,2020-06-21,95.0,0.0,95.0,237.0,21.0,0.0,1.0,0.801688,190.0,5
4798,Imperial,2020-06-22,95.0,0.0,95.0,237.0,17.0,0.0,0.0,0.801688,190.0,5
